In [1]:
import numpy as np
import matplotlib.pyplot as plt
from cellpose import models, io, utils
from cellpose.io import imread
import imageio.v2 as imageio
from imageio.v2 import imread
import skimage as sk
import napari
import pandas as pd
import os

In [72]:
red_path = 'E:/Rothbart_Lab/Kate/MitoTracker/Round_3_Imaging/Split_Channels/C1_MitoTracker_Red'
green_path = 'E:/Rothbart_Lab/Kate/MitoTracker/Round_3_Imaging/Split_Channels/C2_MitoTracker_Green'
red_images = os.listdir(red_path)
green_images = os.listdir(green_path)
save_df_path = 'E:/Rothbart_Lab/Kate/MitoTracker/Round_3_Imaging/Split_Channels/dataframes'
save_imgs_path = 'E:/Rothbart_Lab/Kate/MitoTracker/Round_3_Imaging/Split_Channels/image_stacks'
cp_masks_path = 'E:/Rothbart_Lab/Kate/MitoTracker/Round_3_Imaging/Split_Channels/CP_masks'
print(red_images)
print(green_images)
#create avg projections of each channel
avg_img_red = []
avg_img_green = []
for i in range(len(red_images)):
    img_red = imread(os.path.join(red_path, red_images[i]))
    np_img_red = np.asarray(img_red, dtype=np.float32)
    avg_red = np.average(np_img_red, axis=0)
    avg_img_red.append(avg_red)
    img_green = imread(os.path.join(green_path, green_images[i]))
    np_img_green = np.asarray(img_green, dtype=np.float32)
    avg_green = np.average(np_img_green, axis=0)
    avg_img_green.append(avg_green)
print('Done creating arrays')
#Create a ratio image of red/green
ratio_imgs = []
for i in range(len(avg_img_green)):
    ratio = np.divide(avg_img_red[i], avg_img_green[i], out=np.zeros_like(avg_img_red), where=avg_img_green!=0)
    ratio = ratio[1:]
    ratio = np.squeeze(ratio, axis=0)
    ratio_imgs.append(ratio)
print('Done generating ratio image')
#Send ratio images to CP to generate masks
io.logger_setup()
model = models.Cellpose(gpu=True,model_type='cyto3') # model_type='cyto' or 'nuclei' or 'cyto2' or 'cyto3'
channels =[[0,0]]
masks, flows, styles, diams = model.eval(ratio_imgs, diameter=270, channels=channels)
for i in range(len(masks)):
    sk.io.imsave(os.path.join(cp_masks_path,'ratio_mask_'+red_images[i]),masks[i])
print('Done creating CP masks')
#Stack images and then get measurements
img_stacks = []
for i in range(len(avg_img_green)):
    image_list = [avg_img_green[i],avg_img_red[i],ratio_imgs[i]]
    all_channels = np.stack(image_list, axis=-1)
    img_stacks.append(all_channels)
    print('image ', str(i), ' shape:', all_channels.shape)
    sk.io.imsave(os.path.join(save_imgs_path,'ratio_'+red_images[i]),all_channels)
    cell_masks = sk.measure.label(masks[i]) #does not create a label image in Napari
    ratio_measurements = sk.measure.regionprops_table(cell_masks, img_stacks[i], 
                                                  properties=('label','area', 'centroid','intensity_max','intensity_mean','intensity_min'))
    df = pd.DataFrame.from_dict(ratio_measurements)
    clean_name = str(red_images[i])
    clean_name = clean_name[3:-4]
    df.to_csv(os.path.join(save_df_path,'measurements_'+clean_name+'.csv'))
print('Done with creating measurements!')

['C1-A1_Veh_a_2024_06_21__17_06_32.tif', 'C1-A3_T_a_2024_06_21__15_58_06.tif']
['C2-A1_Veh_a_2024_06_21__17_06_32.tif', 'C2-A3_T_a_2024_06_21__15_58_06.tif']
Done creating arrays
Done generating ratio image


E:\Temp\ipykernel_35072\2443797558.py:37: UserWarning: E:/Rothbart_Lab/Kate/MitoTracker/Round_3_Imaging/Split_Channels/CP_masks\ratio_mask_C1-A1_Veh_a_2024_06_21__17_06_32.tif is a low contrast image
  sk.io.imsave(os.path.join(cp_masks_path,'ratio_mask_'+red_images[i]),masks[i])
E:\Temp\ipykernel_35072\2443797558.py:37: UserWarning: E:/Rothbart_Lab/Kate/MitoTracker/Round_3_Imaging/Split_Channels/CP_masks\ratio_mask_C1-A3_T_a_2024_06_21__15_58_06.tif is a low contrast image
  sk.io.imsave(os.path.join(cp_masks_path,'ratio_mask_'+red_images[i]),masks[i])


Done creating CP masks
image  0  shape: (2048, 2048, 3)
image  1  shape: (2048, 2048, 3)
Done with creating measurements!


In [25]:
#create a loop that generates an avg projections of each image
avg_img_red = []
avg_img_green = []
for i in range(len(red_images)):
    img_red = imread(os.path.join(red_path, red_images[i]))
    np_img_red = np.asarray(img_red, dtype=np.float32)
    avg_red = np.average(np_img_red, axis=0)
    avg_img_red.append(avg_red)
    img_green = imread(os.path.join(green_path, green_images[i]))
    np_img_green = np.asarray(img_green, dtype=np.float32)
    avg_green = np.average(np_img_green, axis=0)
    avg_img_green.append(avg_green)


In [55]:
ratio_imgs = []
for i in range(len(avg_img_green)):
    ratio = np.divide(avg_img_red[i], avg_img_green[i], out=np.zeros_like(avg_img_red), where=avg_img_green!=0)
    ratio = ratio[1:]
    ratio = np.squeeze(ratio, axis=0)
    ratio_imgs.append(ratio)
print(ratio_imgs[0].shape)

(2048, 2048)


In [64]:
#run CP on all ratio images; automatically runs on all images in a list; outputs all masks into a list
io.logger_setup()
# model_type='cyto' or 'nuclei' or 'cyto2' or 'cyto3'
model = models.Cellpose(gpu=True,model_type='cyto3')
channels =[[0,0]]
masks, flows, styles, diams = model.eval(ratio_imgs, diameter=270, channels=channels)

In [65]:
for i in range(len(masks)):
    sk.io.imsave(os.path.join(cp_masks_path,'ratio_mask_'+red_images[i]),masks[i])

E:\Temp\ipykernel_35072\1684865539.py:2: UserWarning: E:/Rothbart_Lab/Kate/MitoTracker/Round_3_Imaging/Split_Channels/CP_masks\ratio_mask_C1-A1_Veh_a_2024_06_21__17_06_32.tif is a low contrast image
  sk.io.imsave(os.path.join(cp_masks_path,'ratio_mask_'+red_images[i]),masks[i])
E:\Temp\ipykernel_35072\1684865539.py:2: UserWarning: E:/Rothbart_Lab/Kate/MitoTracker/Round_3_Imaging/Split_Channels/CP_masks\ratio_mask_C1-A3_T_a_2024_06_21__15_58_06.tif is a low contrast image
  sk.io.imsave(os.path.join(cp_masks_path,'ratio_mask_'+red_images[i]),masks[i])


In [38]:
print('avg green shape:', avg_img_green[0].shape)
print('avg red shape:', avg_img_red[0].shape)
print('ratio shape:', ratio_imgs[0].shape)

avg green shape: (2048, 2048)
avg red shape: (2048, 2048)
ratio shape: (2, 2048, 2048)


In [66]:
#stack images and then get measurements
img_stacks = []
for i in range(len(avg_img_green)):
    image_list = [avg_img_green[i],avg_img_red[i],ratio_imgs[i]]
    all_channels = np.stack(image_list, axis=-1)
    img_stacks.append(all_channels)
    print('image ', str(i), ' shape:', all_channels.shape)
    sk.io.imsave(os.path.join(save_imgs_path,'ratio_'+red_images[i]),all_channels)

image  0  shape: (2048, 2048, 3)
image  1  shape: (2048, 2048, 3)


In [71]:
#mesaure all fluorescence intensities; create label image for directing measurements then measure within those labels
for i in range(len(img_stacks)):
    cell_masks = sk.measure.label(masks[i]) #does not create a label image in Napari
    ratio_measurements = sk.measure.regionprops_table(cell_masks, img_stacks[i], 
                                                  properties=('label','area', 'centroid','intensity_max','intensity_mean','intensity_min'))
    df = pd.DataFrame.from_dict(ratio_measurements)
    clean_name = str(red_images[i])
    clean_name = clean_name[3:-4]
    df.to_csv(os.path.join(save_df_path,'measurements_'+clean_name+'.csv'))
    

In [59]:
viewer = napari.view_image(img_stacks[0], name= 'test')

In [ ]:
io.logger_setup()

# model_type='cyto' or 'nuclei' or 'cyto2' or 'cyto3'
model = models.Cellpose(gpu=True,model_type='cyto3')

# list of files
# PUT PATH TO YOUR FILES HERE!
#file = 'E:/Rothbart_Lab/Kate/MitoTracker/Round_3_Imaging/Split_Channels/Reconstructed_TPMT/MED_top_hat-43125078.tif'

imgs = imread(file)
# nimg = len(imgs)

# define CHANNELS to run segementation on
# grayscale=0, R=1, G=2, B=3
# channels = [cytoplasm, nucleus]
# if NUCLEUS channel does not exist, set the second channel to 0
channels = [[0,0]]
# IF ALL YOUR IMAGES ARE THE SAME TYPE, you can give a list with 2 elements
# channels = [0,0] # IF YOU HAVE GRAYSCALE
# channels = [2,3] # IF YOU HAVE G=cytoplasm and B=nucleus
# channels = [2,1] # IF YOU HAVE G=cytoplasm and R=nucleus

# if diameter is set to None, the size of the cells is estimated on a per image basis
# you can set the average cell `diameter` in pixels yourself (recommended)
# diameter can be a list or a single number for all images

masks, flows, styles, diams = model.eval(imgs, diameter=270, channels=channels)


### or to run one of the other models, or a custom model, specify a CellposeModel
# model = models.CellposeModel(model_type='livecell_cp3')

# masks, flows, styles = model.eval(imgs, diameter=30, channels=[0,0])

In [ ]:
viewer = napari.view_image(imgs, name = 'MED_TopHat')
viewer.add_image(masks, name = 'masks')

In [ ]:
from cellpose import plot
fig = plt.figure(figsize=(12,5))
plot.show_segmentation(fig, imgs, masks, flows[0], channels=channels)
plt.tight_layout()
plt.show()

In [41]:
#Read in the images, and make sure they are visible in Napari
red_channel = imread('E:/Rothbart_Lab/Kate/MitoTracker/Round_3_Imaging/Split_Channels/C1_MitoTracker_Red/C1-A1_Veh_a_2024_06_21__17_06_32.tif')
green_channel = imread('E:/Rothbart_Lab/Kate/MitoTracker/Round_3_Imaging/Split_Channels/C2_MitoTracker_Green/C2-A1_Veh_a_2024_06_21__17_06_32.tif')
viewer = napari.view_image(red_channel, name = 'Mito Red')
viewer.add_image(green_channel, name = 'Mito Green')

<Image layer 'Mito Green' at 0x14119e72b80>

In [42]:
#convert image to numpy array for maths
red_array = np.asarray(red_channel, dtype=np.float32)
green_array = np.asarray(green_channel,dtype=np.float32)

In [ ]:
#cofirm the conversion to float32
print("red array: ", red_array.dtype)
print("green: ", green_array.dtype)

In [43]:
#Creating the ratio array between the red and green channels; maintaining all #/0 as 0
#green_over_red_ratio = np.divide(green_array, red_array, out=np.zeros_like(green_array), where=red_array!=0) #leaves gaps where the cells are
red_over_green_ratio = np.divide(red_array, green_array, out=np.zeros_like(red_array), where=green_array!=0) #out=np.zeros_like(red_array), where=green_array!=0 --> where values are divided by zero, keeps a zero in it's place
print(red_over_green_ratio.shape)

(24, 2048, 2048)


In [ ]:
#what does the ratio look like?
viewer.add_image(ratio, name='otherway_ratio')

In [ ]:
#checking how many 0s are in the arrays
non_zero = np.count_nonzero(red_array)
total = 24*2048*2048
zeros = total - non_zero
print('total: ', total)
print('non-zeros: ', non_zero)
print('zeros: ',zeros)

In [44]:
#creating average projections of the stacks
avg_g = np.average(green_array, axis=0)
#viewer.add_image(avg_green, name = "green_avg_project")
avg_r = np.average(red_array, axis=0)
#viewer.add_image(avg_red, name = "red_avg_project")

In [45]:
#Calculate ratio of red/green from avg projections
red_green_ratio = np.divide(avg_r, avg_g, out=np.zeros_like(avg_r), where=avg_g!=0) #out=np.zeros_like(red_array), where=green_array!=0 --> where values are divided by zero, keeps a zero in it's place
#viewer.add_image(red_green_ratio, name = 'red_green_ratio')
print(red_green_ratio.shape)

(2048, 2048)


In [ ]:
viewer.add_image(masks, name = 'masks')

In [ ]:
#running CellPose on the avg_green mito projection
io.logger_setup()

# model_type='cyto' or 'nuclei' or 'cyto2' or 'cyto3'
model = models.Cellpose(gpu=True,model_type='cyto3')

# list of files
# PUT PATH TO YOUR FILES HERE!

# nimg = len(imgs)

# define CHANNELS to run segementation on
# grayscale=0, R=1, G=2, B=3
# channels = [cytoplasm, nucleus]
# if NUCLEUS channel does not exist, set the second channel to 0
channels = [[0,0]]
# IF ALL YOUR IMAGES ARE THE SAME TYPE, you can give a list with 2 elements
# channels = [0,0] # IF YOU HAVE GRAYSCALE
# channels = [2,3] # IF YOU HAVE G=cytoplasm and B=nucleus
# channels = [2,1] # IF YOU HAVE G=cytoplasm and R=nucleus

# if diameter is set to None, the size of the cells is estimated on a per image basis
# you can set the average cell `diameter` in pixels yourself (recommended)
# diameter can be a list or a single number for all images

masks, flows, styles, diams = model.eval(red_green_ratio, diameter=270, channels=channels)

In [ ]:
viewer.add_image(masks, name = 'masks')

In [ ]:
cell_masks = sk.measure.label(masks) #does not create a label image in Napari
ratio_measurements = sk.measure.regionprops_table(cell_masks, red_green_ratio, 
                                                  properties=('label','area', 'centroid','intensity_max','intensity_mean','intensity_min'))

In [ ]:
image_list = [avg_green, avg_red, red_green_ratio]
all_channels = np.stack(image_list, axis=-1)
all_channels.shape

In [ ]:
ratio_measurements = sk.measure.regionprops_table(cell_masks, all_channels, 
                                                  properties=('label','area', 'centroid','intensity_max','intensity_mean','intensity_min'))

In [ ]:
df = pd.DataFrame.from_dict(ratio_measurements)

In [ ]:
print(df)

In [ ]:
#save the datums!
sk.io.imsave('E:/Rothbart_Lab/Kate/MitoTracker/Round_3_Imaging/Split_Channels/Python_output_test/AVG_Mito_Red.tiff',avg_red)

In [ ]:
sk.io.imsave('E:/Rothbart_Lab/Kate/MitoTracker/Round_3_Imaging/Split_Channels/Python_output_test/AVG_Mito_Green.tiff',avg_green)
sk.io.imsave('E:/Rothbart_Lab/Kate/MitoTracker/Round_3_Imaging/Split_Channels/Python_output_test/Ratio_Red_Green.tiff',red_green_ratio)
sk.io.imsave('E:/Rothbart_Lab/Kate/MitoTracker/Round_3_Imaging/Split_Channels/Python_output_test/CellPose_Masks.tiff',masks)
sk.io.imsave('E:/Rothbart_Lab/Kate/MitoTracker/Round_3_Imaging/Split_Channels/Python_output_test/All_channels.tiff',all_channels)

In [ ]:
df.to_csv('E:/Rothbart_Lab/Kate/MitoTracker/Round_3_Imaging/Split_Channels/Python_output_test/Measurements.csv')

In [ ]:
#turn it into a loop!
